In [1]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'Ams_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

Ams_pipeline


In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))
# ws.get_details()



Ready to use Azure ML 1.43.0 to work with tcp0044aedcmluc001dtmlws


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "tcp0044uc001cc03"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [4]:
%%writefile $experiment_folder/python_env2.yml

name: python_env2

dependencies:
- python
- anaconda
- pip
- pip:
      - azureml.sdk
      - sklearn
      - statsmodels
      - pyarrow
      - numpy
      - pandas
      - seaborn
      - joblib
      - shap
      - graphviz 
      - tensorflow
      - plotly
      - pmdarima
      - emmv
           

Overwriting Ams_pipeline/python_env2.yml


In [5]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import DockerConfiguration
#from azureml.core import ScriptRunConfig

#Create a Python environment for the experiment (from a .yml file)

experiment_env = Environment.from_conda_specification("python_env2", experiment_folder + "/python_env2.yml")


experiment_env.register(workspace=ws)

# registered_env = Environment.get(ws, 'python_env2')
registered_env = Environment.get(ws,name="python_env2",version="18")

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

No Python version provided, defaulting to "3.8.13"


In [6]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

# Get the training dataset
# = ws.datasets.get("")

# Create an OutputFileDatasetConfig (temporary Data Reference) 
#prepped_data = OutputFileDatasetConfig("prepped_data")

# Run the data prep and train script
prep_train_step = PythonScriptStep(name = "train model",
                                source_directory = experiment_folder ,
                                script_name = "AMS_newdata_allmodels_ver2-copy.py",
                              #arguments = ['--input-data'.as_named_input('raw_data'),
                                             #'--prepped-data', prepped_data],
                                #compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

In [7]:
# from azureml.core.authentication import MsiAuthentication

# msi_auth = MsiAuthentication()

# ws = Workspace(subscription_id="49dfc6a3-5fb7-49f4-adea-c54e725bb854",
#                resource_group="tcp-azu0044-ae-rg-dcml-uc001-dt",
#                workspace_name="tcp0044aedcmluc001dtmlws",
#                auth=msi_auth)

# print("Found workspace {} at location {}".format(ws.name, ws.location))





# from azureml.core.authentication import InteractiveLoginAuthentication
# forced_interactive_auth = InteractiveLoginAuthentication(tenant_id="49dfc6a3-5fb7-49f4-adea-c54e725bb854",force=True)

# ws = Workspace(subscription_id="c4f0bb45-07a9-489a-8f6a-6fbc64f25b46",
#                resource_group="tcp-azu0044-ae-rg-dcml-uc001-dt",
#                workspace_name="tcp0044aedcmluc001dtmlws",
#                auth=forced_interactive_auth)

# print("Found workspace {} at location {}".format(ws.name, ws.location))

# from azureml.core.authentication import MsiAuthentication

# msi_auth = MsiAuthentication()

# ws = Workspace(subscription_id="49dfc6a3-5fb7-49f4-adea-c54e725bb854",
#                resource_group="tcp-azu0044-ae-rg-dcml-uc001-dt",
#                workspace_name="tcp0044aedcmluc001dtmlws",
#                auth=msi_auth)

# print("Found workspace {} at location {}".format(ws.name, ws.location))







In [8]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails


# Construct the pipeline
pipeline_steps = [prep_train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps,description="Traning")
print("Pipeline is built.")


# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'AMS_experiment_1')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()

pipeline_run.wait_for_completion(show_output=True)



Pipeline is built.
Created step train model [8dba8fb6][61a4e342-3b92-4cba-aae6-64f3e3240bae], (This step will run and generate new outputs)
Submitted PipelineRun d701e0c0-f7ef-497a-b179-b7ba2623261a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d701e0c0-f7ef-497a-b179-b7ba2623261a?wsid=/subscriptions/c4f0bb45-07a9-489a-8f6a-6fbc64f25b46/resourcegroups/tcp-azu0044-ae-rg-dcml-uc001-dt/workspaces/tcp0044aedcmluc001dtmlws&tid=49dfc6a3-5fb7-49f4-adea-c54e725bb854
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: d701e0c0-f7ef-497a-b179-b7ba2623261a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d701e0c0-f7ef-497a-b179-b7ba2623261a?wsid=/subscriptions/c4f0bb45-07a9-489a-8f6a-6fbc64f25b46/resourcegroups/tcp-azu0044-ae-rg-dcml-uc001-dt/workspaces/tcp0044aedcmluc001dtmlws&tid=49dfc6a3-5fb7-49f4-adea-c54e725bb854

PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': 'd701e0c0-f7ef-497a-b179-b7ba2623261a', 'status': 'Completed', 'startTimeUtc': '2022-07-27T06:53:14.793057Z', 'endTimeUtc': '2022-07-27T06:53:17.159161Z', 'services': {}, 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK', 'azureml.parameters': '{}', 'azureml.continue_on_step_failure': 'False', 'azureml.continue_on_failed_optional_input': 'True', 'azureml.pipelineComponent': 'pipelinerun'}, 'inputDatasets': [], 'outputDatasets': [], 'logFiles': {'logs/azureml/executionlogs.txt': 'https://tcp0044aedcmluc001dtsa.blob.core.windows.net/azureml/Expe

'Finished'

In [9]:
# from azureml.core import Model

# for model in Model.list(ws):
#     print(model.name, 'version:', model.version)
#     for tag_name in model.tags:
#         tag = model.tags[tag_name]
#         print ('\t',tag_name, ':', tag)
#     for prop_name in model.properties:
#         prop = model.properties[prop_name]
#         print ('\t',prop_name, ':', prop)
#     print('\n')

In [10]:
# run_details = pipeline_run.get_details()
# print(run_details)

In [11]:
# # Publish the pipeline from the run
# published_pipeline = pipeline_run.publish_pipeline(
#     name="AMS_anomaly_training_pipeline", description="AMS_all_models", version="1.0")

# published_pipeline

# print(published_pipeline)

In [12]:
# rest_endpoint = published_pipeline.endpoint
# print(rest_endpoint)

In [13]:
# # from azureml.core.authentication import InteractiveLoginAuthentication
# # forced_interactive_auth = InteractiveLoginAuthentication(tenant_id="49dfc6a3-5fb7-49f4-adea-c54e725bb854",force=True)

# # ws = Workspace(subscription_id="c4f0bb45-07a9-489a-8f6a-6fbc64f25b46",
# #                resource_group="tcp-azu0044-ae-rg-dcml-uc001-dt",
# #                workspace_name="tcp0044aedcmluc001dtmlws",
# #                auth=forced_interactive_auth)

# # print("Found workspace {} at location {}".format(ws.name, ws.location))



# from azureml.core.authentication import InteractiveLoginAuthentication

# interactive_auth = InteractiveLoginAuthentication()
# auth_header = interactive_auth.get_authentication_header()
# print("Authentication header ready.")

In [14]:
# import requests

# experiment_name = 'AMS_experiment_1'

# rest_endpoint = published_pipeline.endpoint
# response = requests.post(rest_endpoint, 
#                          headers=auth_header, 
#                          json={"ExperimentName": experiment_name})
# run_id = response.json()["Id"]
# run_id

In [15]:
# from azureml.pipeline.core.run import PipelineRun

# published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
# published_pipeline_run.wait_for_completion(show_output=True)

In [16]:
# from azureml.pipeline.core import ScheduleRecurrence, Schedule

# # Submit the Pipeline every Day at 
# recurrence = ScheduleRecurrence(frequency="Day", interval=1, time_of_day="15:30" )
# day_schedule = Schedule.create(ws, name="daily-AMS_anomaly-training", 
#                                   description="Daily_run",
#                                   pipeline_id=published_pipeline.id, 
#                                   experiment_name='AMS_experiment_1', 
#                                   recurrence=recurrence)
# print('Pipeline scheduled.')






# from azureml.pipeline.core import ScheduleRecurrence, Schedule

# # Submit the Pipeline every Monday at 00:00 UTC
# recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
# weekly_schedule = Schedule.create(ws, name="weekly-diabetes-training", 
#                                   description="Based on time",
#                                   pipeline_id=published_pipeline.id, 
#                                   experiment_name='mslearn-diabetes-pipeline', 
#                                   recurrence=recurrence)
# print('Pipeline scheduled.')

In [17]:
#  from azureml.pipeline.core import Schedule
#  import azureml.core
#  from azureml.core import Workspace
    
#  # Load the workspace from the saved config file
#  ws = Workspace.from_config()
#  sch = Schedule.list(ws)[0] # I want to disable the first pipeline
    
#  Schedule.disable(sch)
#  sch

In [18]:
# sch= Schedule.list(ws)
# sch

In [19]:
# pipeline_experiment = ws.experiments.get('AMS_experiment_1')
# latest_run = list(pipeline_experiment.get_runs())[0]

# latest_run.get_details()